In [1]:
import pandas as pd
from cognite.client.data_classes import functions
from cognite.client.data_classes.functions import FunctionSchedulesList
from cognite.client.data_classes.functions import FunctionSchedule

from initialize import initialize_client
cdf_env = "dev"

# Set limit on function calls - don't think it's really necessary ...
func_limits = functions.FunctionsLimits(timeout_minutes=60, cpu_cores=0.25, memory_gb=1, runtimes=["py39"], response_size_mb=2)
client = initialize_client(cdf_env)

##### Need to store model parameters in file, and upload this to CDF to be able to extract trained parameters for next call to the Cognite Function (fine-tuning for next schedule and prediction for subsequent period)

In [150]:
# client.files.upload("dummy_data.csv", external_id="dummy_data", name="dummy_data.csv", data_set_id=1832663593546318, mime_type="text/csv")
# client.files.download("cf_dummy", external_id="dummy_data")
# client.files.delete(external_id="dummy_data")

In [62]:
import ast
from datetime import datetime, timedelta

# data[col]: prints pd.Series object
# data[[col]]: prints pd.DataFrame object

myfunc = client.functions.retrieve(external_id="cf_power-Template")
my_schedule_id = client.functions.schedules.list(
                name="VAL_11-LT-95107A:X.Value").to_pandas().id[0]
scheduled_calls = myfunc.list_calls(schedule_id=my_schedule_id, limit=-1).to_pandas()
start_time = (datetime(2024, 1, 15, 15, 30) - datetime(1970, 1, 1, 0, 0)).total_seconds()*1000
end_time = (datetime(2024, 1, 15, 15, 45) - datetime(1970, 1, 1, 0, 0)).total_seconds()*1000

mask_start = scheduled_calls["scheduled_time"] >= start_time
mask_end = scheduled_calls["scheduled_time"] < end_time
myid = scheduled_calls[mask_start & mask_end]["id"].iloc[0]
myfunc.retrieve_call(id=myid).get_response() is None
# test = client.functions.calls.retrieve(call_id=3005253751851002, function_id=84587311037983).get_response()

True

In [37]:
client.time_series.data.retrieve(external_id="CF_IdealPowerConsumption",
                                 aggregates="stepInterpolation",
                                 granularity="1h",
                                 limit=100)

,CF_IdealPowerConsumption|step_interpolation
2020-02-16 23:00:00,0.0
2020-02-17 00:00:00,0.0
2020-02-17 01:00:00,0.0
2020-02-17 02:00:00,0.0
2020-02-17 03:00:00,0.0
...,...
2020-02-20 22:00:00,0.0
2020-02-20 23:00:00,0.0
2020-02-21 00:00:00,0.0
2020-02-21 01:00:00,0.0


In [33]:
import pandas as pd
import numpy as np
import json

# Create a DataFrame with three columns, each containing 5 random numeric values
data = {
    'Column1': np.random.rand(5),
    'Column2': np.random.rand(5),
    'Column3': np.random.rand(5)
}

df = pd.DataFrame(data)
inp = {"Column1": [], "Column2": []}
inp["Column1"] = json.dumps(None)
inp["Column1"] = ast.literal_eval(df["Column1"].to_json())
inp["Column2"] = ast.literal_eval(df["Column2"].to_json())
json.dumps(inp)

test = client.time_series.list(
                    name="power_Template").to_pandas()

False

In [57]:
import pandas as pd
import json
test = pd.DataFrame([[1,2,3,4,5], [5,6,7,6,5]]).T
ast.literal_eval('{"test": None}')
orig = ast.literal_eval(test[0].to_json())
ast.literal_eval(json.dumps({"test": orig, "gsgg": json.dumps(None)}))

{'test': {'0': 1, '1': 2, '2': 3, '3': 4, '4': 5}, 'gsgg': 'null'}

In [44]:
from datetime import datetime
import numpy as np
dat = pd.date_range(start=datetime(2023,4,5,0),
                end=datetime(2023,4,6,2),
                freq="S")

df = pd.DataFrame(np.arange(len(dat)), index=dat, columns=["data"])
df.resample("H").agg("mean").iloc[0:20]
# df

,data
2023-04-05 00:00:00,1799.5
2023-04-05 01:00:00,5399.5
2023-04-05 02:00:00,8999.5
2023-04-05 03:00:00,12599.5
2023-04-05 04:00:00,16199.5
2023-04-05 05:00:00,19799.5
2023-04-05 06:00:00,23399.5
2023-04-05 07:00:00,26999.5
2023-04-05 08:00:00,30599.5
2023-04-05 09:00:00,34199.5


sid = client.functions.schedules.list(function_id=func_drainage.id).to_pandas().id[0]
scid = func_drainage.list_calls(schedule_id=sid, limit=-1).to_pandas()
resp = func_drainage.retrieve_call(id=scid).get_response()
resp

my_func = client.functions.retrieve(external_id=data_dict["function_name"])
my_schedule_id = client.functions.schedules.list(
            name=data_dict["function_name"]).to_pandas().id[0]
all_calls = my_func.list_calls(
            schedule_id=my_schedule_id, limit=-1).to_pandas()
all_calls.tail()

pd.date_range(start=datetime(2023,11,16,0,0), end=datetime(2023,11,16,3,51), freq="T")
extid = client.time_series.list(name="VAL_17-FI-9101-286:VALUE")[0].external_id
ts_orig_all = client.time_series.data.retrieve(external_id=extid,
                                                   limit=20,
                                                   ).to_pandas()
ts_orig_all.head()